In [1]:
%pip install sentence_transformers -q gwpy
%pip install open_clip_torch -q gwpy
%pip install torcheval -q gwpy
%pip install clip_client -q gwpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 55.4 MB/s eta 0:00:0

In [2]:
import torch
import gzip
import json
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sentence_transformers import SentenceTransformer
import open_clip
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torcheval.metrics import R2Score
from timeit import default_timer as timer
from clip_client import Client
import numpy as np

In [3]:
os.chdir("/content/drive/MyDrive/DSN_Project/Project")
print(os.getcwd())

/content/drive/MyDrive/DSN_Project/Project


In [4]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
print(device)

cuda


In [5]:
def CLIP_Encode(image_paths):
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
    for i in range(len(image_paths)):
      image_paths[i] = preprocess(Image.open(image_paths[i])).unsqueeze(0)
    image_paths = torch.stack(image_paths)
    image_paths = torch.squeeze(image_paths,1)
    with torch.no_grad():
        image_embeds = model.encode_image(image_paths)

    return image_embeds

def BERT_Encode(sentences):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    #Sentences are encoded by calling model.encode()
    text_embeds = model.encode(sentences)

    return torch.tensor(text_embeds)


class AmazonDataset(Dataset):
    def __init__(self, X, y, embeded=False):
        self._X = X
        self._y = y
        self._embeded = embeded

    def __len__(self):
        return len(self._y)

    def __getitem__(self, index:int):
        if self._embeded:
          return [self._X[index][0],self._X[index][1]], torch.tensor(self._y[index], dtype=torch.float32).to(device)
        else:
          return [self._X[index][0],self._X[index][1]], torch.tensor(self._y[index], dtype=torch.float32).to(device)

    def Embed(self,dataloader):
          X_embed = []
          y_embed = []
          self._embeded = True
          for batch, (X, y) in enumerate(tqdm(dataloader,position=0, leave=True)):
            X_txt = list(X[0])
            X_img = list(X[1])
            X_txt = BERT_Encode(X_txt)
            X_img = CLIP_Encode(X_img)
            #print(X_txt.size(), X_img.size())
            X_concat = torch.cat((X_txt,X_img),1).float().to(device)
            y = y.to(device)
            y = y.view(y.size()[0],1)
            X_embed.append(X_concat)
            y_embed.append(y)

          torch.save(X_embed, "X_tensor.pt")
          torch.save(y_embed, "y_tensor.pt")
          return X_embed, y_embed



def Read(f_data):
    f = open(f_data,'rb')
    data = f.readlines()
    data = json.loads(data[0])
    X = []
    y = []

    for dp in tqdm(data):
        dp = data[dp]
        image_path = f"Images/{dp['asin']}.png"
        try:
            Image.open(image_path)
            X.append([dp['description'],image_path])
            y.append(dp['score'])
        except:
            continue
    return X, y


class CLIP_Classifier(torch.nn.Module):
    def __init__(self, input_dim,
                 output_dim,
                 hidden_dim : int = 256):

        super(CLIP_Classifier,self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

        self.hidden = torch.nn.Linear(self.input_dim, self.hidden_dim)
        self.final = torch.nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, input):
      x = torch.nn.functional.relu(self.hidden(input))
      x = torch.nn.functional.relu(self.final(x))

      return x



In [8]:
#### Execution ####
if os.path.isfile("X_tensor.pt") and os.path.isfile("y_tensor.pt"):
  X_embed = torch.load("X_tensor.pt")
  y_embed = torch.load("y_tensor.pt")
else:
  device = "cuda" if torch.cuda.is_available() else "cpu"
  f_image = os.listdir("Images")

  # CONFIG
  batch_size = 256

  X, y = Read("Fashion_data.txt")

  #### Execution ####
  dataset = AmazonDataset(X,y)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
  X_embed, y_embed = dataset.Embed(dataloader)

In [9]:
#### Execution ####
def split(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split(X_embed, y_embed)

In [34]:
batch_size = 256

def train_step(model: torch.nn.Module,
               #dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):

    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for i in range(len(y_train)):
        # Send data to target device
        #print(X.shape)
        X = X_train[i]
        y = y_train[i]

        X = X.to(device)
        y = y.to(device)
        y = y.view(y.size()[0],1)


        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        metric = R2Score()
        metric.update(y_pred,y)
        r2 = metric.compute()
        train_acc += r2.item()

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(y_train)
    train_acc = train_acc / len(y_train)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              #dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):

    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
          for i in range(len(y_test)):
            # Send data to target device
            #print(X.shape)
            X = X_test[i]
            y = y_test[i]

            X = X.to(device)
            y = y.to(device)
            y = y.view(y.size()[0],1)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            metric = R2Score()
            metric.update(test_pred_logits,y)
            r2 = metric.compute()
            test_acc += r2.item()

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(y_test)
    test_acc = test_acc / len(y_test)
    return test_loss, test_acc


def train(model: torch.nn.Module,
          #train_dataloader: torch.utils.data.DataLoader,
          #test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          ) :

    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Find Checkpoint
    if os.path.isfile("model_state_dict.pth"):
        if device == 'cpu':
            checkpoint = torch.load("model_state_dict.pth",map_location=torch.device('cpu'))
        else:
            checkpoint = torch.load("model_state_dict.pth")
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        model.train()

        if checkpoint['epoch']:
            checkpoint_epoch = checkpoint['epoch']
    else:
        checkpoint_epoch = 0
    print(f"Starting at epoch {checkpoint_epoch}")


    test_loss, test_acc = 0, 0

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(checkpoint_epoch,epochs),position=0, leave=True):
        train_loss, train_acc = train_step(model=model,
                                          #dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)

        test_loss, test_acc = test_step(model=model,
            #dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "model_state_dict.pth")

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


    # Return the filled results at the end of the epochs
    return results


In [36]:
if __name__ == '__main__':
  model = CLIP_Classifier(input_dim = 896, output_dim=1).to(device)
  loss_fn = torch.nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.05)


  # Setup training and save the results
  start_time = timer()

  results = train(model=model,
                      #train_dataloader=train_dataloader,
                      #test_dataloader=test_dataloader,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      epochs=500,
                      device=device)

  # End the timer and print out how long it took
  end_time = timer()
  print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

Starting at epoch 299


  0%|          | 1/201 [00:00<01:09,  2.89it/s]

Epoch: 300 | train_loss: 1.3681 | train_acc: 0.1017 | test_loss: 1.4253 | test_acc: 0.0694


  1%|          | 2/201 [00:00<01:13,  2.71it/s]

Epoch: 301 | train_loss: 1.3682 | train_acc: 0.1017 | test_loss: 1.4246 | test_acc: 0.0699


...

Epoch: 498 | train_loss: 1.3662 | train_acc: 0.1030 | test_loss: 1.4207 | test_acc: 0.0724


100%|█████████▉| 200/201 [01:01<00:00,  3.22it/s]

Epoch: 499 | train_loss: 1.3663 | train_acc: 0.1029 | test_loss: 1.4205 | test_acc: 0.0725


100%|██████████| 201/201 [01:01<00:00,  3.26it/s]

Epoch: 500 | train_loss: 1.3663 | train_acc: 0.1029 | test_loss: 1.4206 | test_acc: 0.0724
[INFO] Total training time: 61.610 seconds


In [39]:
torch.save(model,"CLIP_Baseline.pth")